In [ ]:
import tensorflow as tf

In [ ]:
file_url = "https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_all_data.zip"

In [ ]:
zip_dir = tf.keras.utils.get_file('food10.zip', origin=file_url, extract=True)

In [ ]:
import pathlib

In [ ]:
path = pathlib.Path(zip_dir).parent / '10_food_classes_all_data'

In [ ]:
train_dir = path / 'train'
validation_dir = path / 'test'

In [ ]:
train_chicken_curry_dir = train_dir / 'chicken_curry'
train_chicken_wings_dir = train_dir /'chicken_wings'
train_fried_rice_dir = train_dir /'fried_rice'
train_grilled_salmon_dir = train_dir /'grilled_salmon'
train_hamburger_dir = train_dir /'hamburger'
train_ice_cream_dir = train_dir /'ice_cream'
train_pizza_dir = train_dir /'pizza'
train_ramen_dir = train_dir /'ramen'
train_steak_dir = train_dir /'steak'
train_sushi_dir = train_dir /'sushi'

validation_chicken_curry_dir = validation_dir / 'chicken_curry'
validation_chicken_wings_dir = validation_dir /'chicken_wings'
validation_fried_rice_dir = validation_dir /'fried_rice'
validation_grilled_salmon_dir = validation_dir /'grilled_salmon'
validation_hamburger_dir = validation_dir /'hamburger'
validation_ice_cream_dir = validation_dir /'ice_cream'
validation_pizza_dir = validation_dir /'pizza'
validation_ramen_dir = validation_dir /'ramen'
validation_steak_dir = validation_dir /'steak'
validation_sushi_dir = validation_dir /'sushi'

In [ ]:
total_train = 7500
total_val = 2500

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_image_generator = ImageDataGenerator(rescale=1./255,
                                          rotation_range=40,
                                          width_shift_range=0.1,
                                          height_shift_range=0.1,
                                          shear_range=0.2,
                                          zoom_range=0.2,
                                          horizontal_flip=True,
                                          fill_mode='nearest')
validation_image_generator = ImageDataGenerator(rescale=1./255)

In [ ]:
batch_size = 32
img_height = 224
img_width = 224
channel = 3

In [ ]:
train_data_gen = train_image_generator.flow_from_directory(
    batch_size = batch_size,
    directory = train_dir,
    shuffle=True,
    target_size = (img_height, img_width),
    class_mode='categorical')

In [ ]:
val_data_gen = validation_image_generator.flow_from_directory(batch_size = batch_size,
                                                              directory = validation_dir,
                                                              target_size=(img_height, img_width),
                                                              class_mode='categorical')

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

In [ ]:
np.random.seed(8)
tf.random.set_seed(8)

In [ ]:
from tensorflow.keras.applications import NASNetMobile

In [ ]:
base_model = NASNetMobile(input_shape=(img_height, img_width, channel), 
                          weights='imagenet', include_top=False)

In [ ]:
base_model.summary()

In [ ]:
model = tf.keras.Sequential([base_model,layers.Flatten(),
                             layers.Dense(1000, activation='relu'), 
                             layers.Dense(10,activation='softmax')])

In [ ]:
optimizer = tf.keras.optimizers.Adam(0.001)

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer, metrics=['accuracy'])

In [ ]:
model.fit(train_data_gen,
          steps_per_epoch=total_train // batch_size,
          epochs=5,
          validation_data=val_data_gen,
          validation_steps=total_val // batch_size)